In [1]:
# Import important libraries
import os
import pandas as pd
from datetime import datetime

In [2]:
# Copy folder path containing all the csv files and open each file using os library
# You can also choose to individually copy each file's path in a list named file_paths. 
folder_path = (r"C:\Users\This pc\Downloads\milkman_admin_sales_report_85_08_10_2023.921282")
file_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.csv')]

In [3]:
# Creating a dataframe to read/load all files and combining them using concat
# Ignore_index = True means the integer indexing will be done from 0 normally (otherwise indexing of combined files would apply)
dfs = [pd.read_csv(file) for file in file_paths]
data = pd.concat(dfs, ignore_index = True)

In [4]:
# JUST TO CHECK PROPERTIES OF OUR DATA
# data.head()
# data.shape
# data['Customer ID'].nunique()
data.isnull().sum()

Order ID                      0
Subscription ID               0
Hub Name                      0
Customer ID                   0
Name                          0
Phone                         0
Latitude                     24
Longitude                    24
Address                     204
Location                      0
Subscription Type           888
Product                       0
Quantity                      0
HSN/SAC Code              50207
Customer Type                 0
SKU Code                      0
Cess                          0
IGST                          0
CGST                          0
SGST                          0
Total Tax                     0
Unit Price Without Tax        0
MRP(Rs)                       0
Unit Price(Rs)                0
Handling Charges              0
Amount(Rs)                    0
Delivery Date                 0
Delivery Time                 0
Marked Delivered By          21
Delivery Executive            0
Account Manager           44730
dtype: i

In [5]:
# Since none of the columns with null values will hamper our RFM score, we will let them be!
# It's just a warning, nothing to worry!
data['Delivery Date'] = pd.to_datetime(data['Delivery Date'])
current_date = pd.to_datetime('2023-08-10')

C:\Users\This pc\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '31-01-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\This pc\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '30-01-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\This pc\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '29-01-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\This pc\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '28-01-2023' in DD/MM/YYYY format. Provide format o

In [9]:
# Calculate Recency, Frequency, and Monetary values
data['Recency'] = (current_date - data.groupby('Customer ID')['Delivery Date'].transform('max')).dt.days
data['Frequency'] = data.groupby('Customer ID')['Order ID'].transform('count')
data['Monetary'] = data.groupby('Customer ID')['Amount(Rs)'].transform('sum')

# Define scoring functions
# The values taken were pre-defined by the organizations. If you have not been specified with the values, consider performing
# normalization first!
def recency_score(x):
    if x <= 14:    #Since we consider data older than 14 days to not be recent
        return 5
    elif x <= 28:
        return 4
    elif x <= 60:
        return 3
    elif x <= 90:
        return 2
    else:
        return 1

def frequency_score(x):
    if x >= 80:
        return 5
    elif x >= 60:
        return 4
    elif x >= 40:
        return 3
    elif x >= 20:
        return 2
    else:
        return 1
    
def monetary_score(x):
    if x >= 900:
        return 5
    elif x >= 600:
        return 4
    elif x >= 300:
        return 3
    elif x >= 100:
        return 2
    else:
        return 1

# Apply scoring functions to create RFM scores
data['RecencyScore'] = data['Recency'].apply(recency_score)
data['FrequencyScore'] = data['Frequency'].apply(frequency_score)
data['MonetaryScore'] = data['Monetary'].apply(monetary_score)

# Calculate RFM Score
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Define segmentation function
def segment(x):
    if x >= 7:
        return 'Premium'
    elif x >= 5:
        return 'Medium'
    else:
        return 'Low'

# Apply segmentation function to create RFM segments
data['RFM_Segment'] = data['RFM_Score'].apply(segment)

# Print the resulting DataFrame
print(data.head())

   Order ID  Subscription ID  Hub Name  Customer ID                Name  \
0       465                3  3217NDA1          155              Bharat   
1       463              481  3217NDA1          191   INDER SINGH BISHT   
2       462              478  3217NDA1          189      URVASHI SAXENA   
3       461              459  3216GUR1          181        NITIN TRIKHA   
4       460              463  3216GUR1          180  ASHUTOSH KUMAR JHA   

        Phone   Latitude  Longitude  \
0  9999645815  28.567836  77.379930   
1  9717263388  28.572527  77.341926   
2  9811897323  28.595123  77.380461   
3  9818116213  28.399981  76.988703   
4  7718874316  28.443915  77.100917   

                                             Address  \
0                    E6, 6, ANTRIKSH GREEN Society,    
1                         C-3/175, Sector 31 Noida,    
2  FLAT NO.E 303,PANOASIS SOCIETY SECTOR 70 NOIDA...   
3     "G-102, DLF New Town, Heights, Sector- 90, ",    
4                          GH-1-2B